In [41]:
from pathlib import Path

import zarr
from micro_sam.training import default_sam_loader
from ngio import open_ome_zarr_container


In [42]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


In [43]:
# Paths
zarr_path = Path(
    r"W:\groups\scratch\gmicro_prefect\ggrossha\ggrossha_SWI\training_data\accumulated_train.zarr"
)
cont = open_ome_zarr_container(zarr_path)
print(zarr.open(str(zarr_path), mode="r").tree())

/
 ├── 0 (3426, 1200, 1200) uint16
 ├── labels
 │   └── mask
 │       └── 0 (3426, 1200, 1200) uint32
 └── tables


In [ ]:
raw_key = "0"
label_key = "labels/mask/0"
loader = default_sam_loader(
    raw_paths=str(zarr_path),
    raw_key=raw_key,
    label_paths=str(zarr_path),
    label_key=label_key,
    is_train=True,
    patch_shape=[2048, 2048],
    with_segmentation_decoder=True,
    batch_size=1,
)

## Test Config Validation

In [ ]:
from sam_trainer.config import TrainingConfig

# Test config with zarr mode
config_dict = {
    "images_dir": str(zarr_path),
    "labels_dir": str(zarr_path),
    "raw_key": "0",
    "label_key": "labels/mask/0",
    "model_type": "vit_b_lm",
    "patch_shape": [2048, 2048],
    "batch_size": 1,
    "n_epochs": 10,
    "checkpoint_name": "test_zarr",
}

config = TrainingConfig(**config_dict)
print(f"Zarr mode: {config.is_zarr_mode()}")
print(f"Config valid: {config}")

ValidationError: 2 validation errors for TrainingConfig
raw_key
  Extra inputs are not permitted [type=extra_forbidden, input_value='0', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden
label_key
  Extra inputs are not permitted [type=extra_forbidden, input_value='labels/mask/0', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

## Test Array Discovery and Splitting

In [ ]:
from sam_trainer.training import prepare_zarr_array_splits

# Test array splitting
train_keys, val_keys = prepare_zarr_array_splits(
    zarr_path,
    raw_key="0",
    label_key="labels/mask/0",
    val_split=0.1,
    shuffle=True,
    seed=42,
)

print(f"Train arrays: {len(train_keys)}")
print(f"Val arrays: {len(val_keys)}")
print(f"First 3 train keys: {train_keys[:3]}")
print(f"First 3 val keys: {val_keys[:3]}")